# 04 - 实时预测模块

此 notebook 负责使用训练好的模型对当前市场数据进行价格走势预测。
包括数据获取、特征准备、模型预测和结果解释。

In [ ]:
import sys
from pathlib import Path
import logging
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt

# 配置绘图
plt.style.use('seaborn-v0_8')
%matplotlib inline

# 配置日志
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()]
)

# 正确的路径设置
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

print(f"\n项目根目录: {project_root}")

预测环境设置完成
项目根目录: /Users/anthony/Documents/github/technial_analysis_helper


In [3]:
# 检查模型可用性
from src.models.xgboost_trainer import xgb_trainer
from src.config.settings import config

print("正在检查模型可用性...")

try:
    # 尝试加载模型
    model_loaded = xgb_trainer.load_model()
    
    if model_loaded:
        print("✓ 模型加载成功")
        print(f"模型文件: {config.MODEL_SAVE_PATH}")
        
        # 显示模型信息
        if hasattr(xgb_trainer, 'model') and xgb_trainer.model is not None:
            print(f"模型类型: {type(xgb_trainer.model).__name__}")
            if hasattr(xgb_trainer.model, 'n_estimators'):
                print(f"树的数量: {xgb_trainer.model.n_estimators}")
    else:
        print("✗ 无法加载模型")
        print("请先运行 03_model_training.ipynb 训练模型")
        
except Exception as e:
    print(f"✗ 检查模型时出错: {e}")

ModuleNotFoundError: No module named 'src'

In [ ]:
# 准备预测用的特征
print("正在准备预测用的特征集...")

if 'df_raw' in locals() and not df_raw.empty:
    try:
        # 确保有足够的数据用于预测
        if len(df_raw) >= config.FEATURE_WINDOW_SIZE:
            # 使用最近的数据
            recent_data = df_raw.tail(config.FEATURE_WINDOW_SIZE).to_dict('records')
            
            # 准备预测特征
            pred_features_df = feature_engineer.prepare_prediction_features(recent_data)
            
            if pred_features_df is not None and not pred_features_df.empty:
                print(f"✓ 预测特征准备完成")
                print(f"  特征数量: {len(pred_features_df.columns)}")
                print(f"  样本数量: {len(pred_features_df)}")
                
                # 显示关键特征
                print(f"\n关键预测特征:")
                key_features = ['current_price', 'short_rsi', 'medium_rsi', 'long_rsi', 
                               'price_volatility', 'volume_avg']
                
                for feature in key_features:
                    if feature in pred_features_df.columns:
                        value = pred_features_df[feature].iloc[0]
                        if isinstance(value, float):
                            if 'price' in feature:
                                print(f"  {feature}: ${value:.2f}")
                            else:
                                print(f"  {feature}: {value:.4f}")
                        else:
                            print(f"  {feature}: {value}")
                
                # 保存预测特征
                pred_features_df.to_csv('prediction_features.csv', index=False)
                print("\n✓ 预测特征已保存为 CSV 文件")
                
            else:
                print("✗ 无法准备预测特征")
        else:
            print(f"✗ 数据不足，需要至少 {config.FEATURE_WINDOW_SIZE} 条记录用于预测")
            
    except Exception as e:
        print(f"✗ 准备预测特征时出错: {e}")
else:
    print("没有可用的数据")

正在准备预测用的特征集...
✓ 预测特征准备完成
  特征数量: 22
  样本数量: 1

关键预测特征:
  current_price: $3134.83
  short_rsi: 60.1009
  medium_rsi: 57.8208
  long_rsi: 57.7885
  price_volatility: $0.00
  volume_avg: 1059939.3288

✓ 预测特征已保存为 CSV 文件


In [ ]:
# 准备预测用的特征
print("正在准备预测用的特征集...")

if 'df_raw' in locals() and not df_raw.empty:
    try:
        # 确保有足够的数据用于预测
        if len(df_raw) >= config.FEATURE_WINDOW_SIZE:
            # 使用最近的数据
            recent_data = df_raw.tail(config.FEATURE_WINDOW_SIZE).to_dict('records')
            
            # 准备预测特征
            pred_features_df = feature_engineer.prepare_prediction_features(recent_data)
            
            if pred_features_df is not None and not pred_features_df.empty:
                print(f"✓ 预测特征准备完成")
                print(f"  特征数量: {len(pred_features_df.columns)}")
                print(f"  样本数量: {len(pred_features_df)}")
                
                # 显示关键特征
                print(f"\n关键预测特征:")
                key_features = ['current_price', 'short_rsi', 'medium_rsi', 'long_rsi', 
                               'price_volatility', 'volume_avg']
                
                for feature in key_features:
                    if feature in pred_features_df.columns:
                        value = pred_features_df[feature].iloc[0]
                        if isinstance(value, float):
                            if 'price' in feature:
                                print(f"  {feature}: ${value:.2f}")
                            else:
                                print(f"  {feature}: {value:.4f}")
                        else:
                            print(f"  {feature}: {value}")
                
                # 保存预测特征
                pred_features_df.to_csv('prediction_features.csv', index=False)
                print("\n✓ 预测特征已保存为 CSV 文件")
                
            else:
                print("✗ 无法准备预测特征")
        else:
            print(f"✗ 数据不足，需要至少 {config.FEATURE_WINDOW_SIZE} 条记录用于预测")
            
    except Exception as e:
        print(f"✗ 准备预测特征时出错: {e}")
else:
    print("没有可用的数据")

正在准备预测用的特征集...
✓ 预测特征准备完成
  特征数量: 22
  样本数量: 1

关键预测特征:
  current_price: $3134.83
  short_rsi: 60.1009
  medium_rsi: 57.8208
  long_rsi: 57.7885
  price_volatility: $0.00
  volume_avg: 1059939.3288

✓ 预测特征已保存为 CSV 文件


In [ ]:
# 准备预测用的特征
print("正在准备预测用的特征集...")

if 'df_raw' in locals() and not df_raw.empty:
    try:
        # 确保有足够的数据用于预测
        if len(df_raw) >= config.FEATURE_WINDOW_SIZE:
            # 使用最近的数据
            recent_data = df_raw.tail(config.FEATURE_WINDOW_SIZE).to_dict('records')
            
            # 准备预测特征
            pred_features_df = feature_engineer.prepare_prediction_features(recent_data)
            
            if pred_features_df is not None and not pred_features_df.empty:
                print(f"✓ 预测特征准备完成")
                print(f"  特征数量: {len(pred_features_df.columns)}")
                print(f"  样本数量: {len(pred_features_df)}")
                
                # 显示关键特征
                print(f"\n关键预测特征:")
                key_features = ['current_price', 'short_rsi', 'medium_rsi', 'long_rsi', 
                               'price_volatility', 'volume_avg']
                
                for feature in key_features:
                    if feature in pred_features_df.columns:
                        value = pred_features_df[feature].iloc[0]
                        if isinstance(value, float):
                            if 'price' in feature:
                                print(f"  {feature}: ${value:.2f}")
                            else:
                                print(f"  {feature}: {value:.4f}")
                        else:
                            print(f"  {feature}: {value}")
                
                # 保存预测特征
                pred_features_df.to_csv('prediction_features.csv', index=False)
                print("\n✓ 预测特征已保存为 CSV 文件")
                
            else:
                print("✗ 无法准备预测特征")
        else:
            print(f"✗ 数据不足，需要至少 {config.FEATURE_WINDOW_SIZE} 条记录用于预测")
            
    except Exception as e:
        print(f"✗ 准备预测特征时出错: {e}")
else:
    print("没有可用的数据")

正在准备预测用的特征集...
✓ 预测特征准备完成
  特征数量: 22
  样本数量: 1

关键预测特征:
  current_price: $3134.83
  short_rsi: 60.1009
  medium_rsi: 57.8208
  long_rsi: 57.7885
  price_volatility: $0.00
  volume_avg: 1059939.3288

✓ 预测特征已保存为 CSV 文件


In [ ]:
# 准备预测用的特征
print("正在准备预测用的特征集...")

if 'df_raw' in locals() and not df_raw.empty:
    try:
        # 确保有足够的数据用于预测
        if len(df_raw) >= config.FEATURE_WINDOW_SIZE:
            # 使用最近的数据
            recent_data = df_raw.tail(config.FEATURE_WINDOW_SIZE).to_dict('records')
            
            # 准备预测特征
            pred_features_df = feature_engineer.prepare_prediction_features(recent_data)
            
            if pred_features_df is not None and not pred_features_df.empty:
                print(f"✓ 预测特征准备完成")
                print(f"  特征数量: {len(pred_features_df.columns)}")
                print(f"  样本数量: {len(pred_features_df)}")
                
                # 显示关键特征
                print(f"\n关键预测特征:")
                key_features = ['current_price', 'short_rsi', 'medium_rsi', 'long_rsi', 
                               'price_volatility', 'volume_avg']
                
                for feature in key_features:
                    if feature in pred_features_df.columns:
                        value = pred_features_df[feature].iloc[0]
                        if isinstance(value, float):
                            if 'price' in feature:
                                print(f"  {feature}: ${value:.2f}")
                            else:
                                print(f"  {feature}: {value:.4f}")
                        else:
                            print(f"  {feature}: {value}")
                
                # 保存预测特征
                pred_features_df.to_csv('prediction_features.csv', index=False)
                print("\n✓ 预测特征已保存为 CSV 文件")
                
            else:
                print("✗ 无法准备预测特征")
        else:
            print(f"✗ 数据不足，需要至少 {config.FEATURE_WINDOW_SIZE} 条记录用于预测")
            
    except Exception as e:
        print(f"✗ 准备预测特征时出错: {e}")
else:
    print("没有可用的数据")

正在准备预测用的特征集...
✓ 预测特征准备完成
  特征数量: 22
  样本数量: 1

关键预测特征:
  current_price: $3134.83
  short_rsi: 60.1009
  medium_rsi: 57.8208
  long_rsi: 57.7885
  price_volatility: $0.00
  volume_avg: 1059939.3288

✓ 预测特征已保存为 CSV 文件


In [ ]:
# 准备预测用的特征
print("正在准备预测用的特征集...")

if 'df_raw' in locals() and not df_raw.empty:
    try:
        # 确保有足够的数据用于预测
        if len(df_raw) >= config.FEATURE_WINDOW_SIZE:
            # 使用最近的数据
            recent_data = df_raw.tail(config.FEATURE_WINDOW_SIZE).to_dict('records')
            
            # 准备预测特征
            pred_features_df = feature_engineer.prepare_prediction_features(recent_data)
            
            if pred_features_df is not None and not pred_features_df.empty:
                print(f"✓ 预测特征准备完成")
                print(f"  特征数量: {len(pred_features_df.columns)}")
                print(f"  样本数量: {len(pred_features_df)}")
                
                # 显示关键特征
                print(f"\n关键预测特征:")
                key_features = ['current_price', 'short_rsi', 'medium_rsi', 'long_rsi', 
                               'price_volatility', 'volume_avg']
                
                for feature in key_features:
                    if feature in pred_features_df.columns:
                        value = pred_features_df[feature].iloc[0]
                        if isinstance(value, float):
                            if 'price' in feature:
                                print(f"  {feature}: ${value:.2f}")
                            else:
                                print(f"  {feature}: {value:.4f}")
                        else:
                            print(f"  {feature}: {value}")
                
                # 保存预测特征
                pred_features_df.to_csv('prediction_features.csv', index=False)
                print("\n✓ 预测特征已保存为 CSV 文件")
                
            else:
                print("✗ 无法准备预测特征")
        else:
            print(f"✗ 数据不足，需要至少 {config.FEATURE_WINDOW_SIZE} 条记录用于预测")
            
    except Exception as e:
        print(f"✗ 准备预测特征时出错: {e}")
else:
    print("没有可用的数据")

正在准备预测用的特征集...
✓ 预测特征准备完成
  特征数量: 22
  样本数量: 1

关键预测特征:
  current_price: $3134.83
  short_rsi: 60.1009
  medium_rsi: 57.8208
  long_rsi: 57.7885
  price_volatility: $0.00
  volume_avg: 1059939.3288

✓ 预测特征已保存为 CSV 文件


In [ ]:
# 准备预测用的特征
print("正在准备预测用的特征集...")

if 'df_raw' in locals() and not df_raw.empty:
    try:
        # 确保有足够的数据用于预测
        if len(df_raw) >= config.FEATURE_WINDOW_SIZE:
            # 使用最近的数据
            recent_data = df_raw.tail(config.FEATURE_WINDOW_SIZE).to_dict('records')
            
            # 准备预测特征
            pred_features_df = feature_engineer.prepare_prediction_features(recent_data)
            
            if pred_features_df is not None and not pred_features_df.empty:
                print(f"✓ 预测特征准备完成")
                print(f"  特征数量: {len(pred_features_df.columns)}")
                print(f"  样本数量: {len(pred_features_df)}")
                
                # 显示关键特征
                print(f"\n关键预测特征:")
                key_features = ['current_price', 'short_rsi', 'medium_rsi', 'long_rsi', 
                               'price_volatility', 'volume_avg']
                
                for feature in key_features:
                    if feature in pred_features_df.columns:
                        value = pred_features_df[feature].iloc[0]
                        if isinstance(value, float):
                            if 'price' in feature:
                                print(f"  {feature}: ${value:.2f}")
                            else:
                                print(f"  {feature}: {value:.4f}")
                        else:
                            print(f"  {feature}: {value}")
                
                # 保存预测特征
                pred_features_df.to_csv('prediction_features.csv', index=False)
                print("\n✓ 预测特征已保存为 CSV 文件")
                
            else:
                print("✗ 无法准备预测特征")
        else:
            print(f"✗ 数据不足，需要至少 {config.FEATURE_WINDOW_SIZE} 条记录用于预测")
            
    except Exception as e:
        print(f"✗ 准备预测特征时出错: {e}")
else:
    print("没有可用的数据")

正在准备预测用的特征集...
✓ 预测特征准备完成
  特征数量: 22
  样本数量: 1

关键预测特征:
  current_price: $3134.83
  short_rsi: 60.1009
  medium_rsi: 57.8208
  long_rsi: 57.7885
  price_volatility: $0.00
  volume_avg: 1059939.3288

✓ 预测特征已保存为 CSV 文件


In [ ]:
# 获取当前市场数据
from src.data.okex_fetcher import okex_fetcher
from src.data.mongodb_handler import mongo_handler

print("正在获取当前市场数据...")

try:
    # 步骤1: 获取最近的实时数据
    print("步骤1: 获取最近的 K 线数据")
    recent_raw_data = okex_fetcher.fetch_candlesticks(bar="1H")
    
    if recent_raw_data:
        recent_data = okex_fetcher._process_candlestick_data(recent_raw_data)
        print(f"✓ 获取到 {len(recent_data)} 条最新数据")
        
        # 显示最新价格信息
        latest_candle = recent_data[0]
        latest_time = datetime.fromtimestamp(latest_candle['timestamp'] / 1000)
        print(f"最新数据时间: {latest_time}")
        print(f"最新收盘价: ${latest_candle['close']:.2f}")
        
    else:
        print("✗ 无法获取实时数据")
        recent_data = []
        
    # 步骤2: 从数据库获取历史数据补充
    print("\n步骤2: 从数据库获取历史数据")
    
    if mongo_handler.connect():
        # 计算需要的历史数据量
        required_history = config.FEATURE_WINDOW_SIZE
        db_data = mongo_handler.get_candlestick_data(limit=required_history)
        
        if db_data:
            print(f"✓ 从数据库获取 {len(db_data)} 条历史数据")
            
            # 转换数据格式
            historical_data = []
            for doc in db_data:
                # 避免与实时数据重复
                if not any(candle['timestamp'] == doc['timestamp'] for candle in recent_data):
                    historical_data.append({
                        'timestamp': doc['timestamp'],
                        'open': doc['open'],
                        'high': doc['high'],
                        'low': doc['low'],
                        'close': doc['close'],
                        'volume': doc['volume']
                    })
            
            print(f"去重后历史数据: {len(historical_data)} 条")
        else:
            print("✗ 数据库中没有历史数据")
            historical_data = []
            
        mongo_handler.close()
    else:
        print("✗ 无法连接到数据库")
        historical_data = []
    
    # 步骤3: 合并所有数据
    print("\n步骤3: 合并数据")
    all_data = historical_data + recent_data
    
    # 按时间排序
    all_data.sort(key=lambda x: x['timestamp'])
    
    print(f"合并后总数据量: {len(all_data)} 条")
    
    if len(all_data) > 0:
        first_time = datetime.fromtimestamp(all_data[0]['timestamp'] / 1000)
        last_time = datetime.fromtimestamp(all_data[-1]['timestamp'] / 1000)
        print(f"数据时间范围: {first_time} 到 {last_time}")
        
        # 显示价格统计
        prices = [candle['close'] for candle in all_data]
        print(f"价格范围: ${min(prices):.2f} - ${max(prices):.2f}")
        print(f"最新价格: ${prices[-1]:.2f}")
        
    else:
        print("✗ 没有可用的数据")
        
except Exception as e:
    print(f"✗ 获取市场数据时出错: {e}")

In [ ]:
# 准备预测特征
from src.utils.feature_engineering import feature_engineer

print("正在准备预测特征...")

if 'all_data' in locals() and len(all_data) >= config.FEATURE_WINDOW_SIZE:
    try:
        # 使用最近的特征窗口数据
        recent_data_window = all_data[-config.FEATURE_WINDOW_SIZE:]
        
        print(f"使用最近 {len(recent_data_window)} 条数据进行特征提取")
        
        # 准备预测特征
        features_df = feature_engineer.prepare_prediction_features(recent_data_window)
        
        if features_df is not None and not features_df.empty:
            print(f"✓ 特征准备完成，共 {len(features_df.columns)} 个特征")
            
            # 显示关键特征
            print(f"\n关键特征值:")
            key_features = ['current_price', 'short_rsi', 'medium_rsi', 'long_rsi', 
                           'price_volatility', 'volume_avg', 'price_trend_24h']
            
            for feature in key_features:
                if feature in features_df.columns:
                    value = features_df[feature].iloc[0]
                    if isinstance(value, float):
                        if 'price' in feature:
                            print(f"  {feature}: ${value:.2f}")
                        elif 'rsi' in feature or 'volatility' in feature:
                            print(f"  {feature}: {value:.4f}")
                        else:
                            print(f"  {feature}: {value:.2f}")
                    else:
                        print(f"  {feature}: {value}")
                        
            # 检查缺失值
            missing_count = features_df.isnull().sum().sum()
            if missing_count > 0:
                print(f"\n⚠️ 发现 {missing_count} 个缺失值，正在进行处理...")
                
                # 填充缺失值
                numeric_columns = features_df.select_dtypes(include=[np.number]).columns
                for col in numeric_columns:
                    if features_df[col].isnull().any():
                        median_value = features_df[col].median()
                        features_df[col].fillna(median_value, inplace=True)
                
                remaining_missing = features_df.isnull().sum().sum()
                print(f"处理后剩余缺失值: {remaining_missing}")
                
        else:
            print("✗ 无法准备预测特征")
            
    except Exception as e:
        print(f"✗ 准备预测特征时出错: {e}")
        
elif 'all_data' in locals():
    required = config.FEATURE_WINDOW_SIZE
    available = len(all_data)
    print(f"✗ 数据不足，需要 {required} 条，当前只有 {available} 条")
    print("请先运行 01_data_acquisition.ipynb 获取更多历史数据")
else:
    print("没有可用的数据")

In [ ]:
# 进行预测
print("正在进行价格走势预测...")

if ('features_df' in locals() and not features_df.empty and 
    'model_loaded' in locals() and model_loaded):
    
    try:
        # 移除不需要的列（如 timestamp）
        prediction_features = features_df.copy()
        if 'timestamp' in prediction_features.columns:
            prediction_features = prediction_features.drop('timestamp', axis=1)
        
        # 进行预测
        predictions, probabilities = xgb_trainer.predict(prediction_features)
        
        # 获取预测结果
        predicted_class = int(predictions[0])
        class_probabilities = probabilities[0]
        
        # 获取置信度
        # 注意：类别编码可能是 -4 到 3，但概率数组索引是 0-7
        predicted_class_index = predicted_class + 4  # 转换为数组索引
        confidence = float(class_probabilities[predicted_class_index])
        
        print(f"\n=== 预测结果 ===")
        print(f"预测时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"当前价格: ${features_df['current_price'].iloc[0]:.2f}")
        print(f"预测类别: {predicted_class}")
        print(f"预测置信度: {confidence:.2%}")
        
        # 获取价格变动区间
        if predicted_class in config.CLASSIFICATION_THRESHOLDS:
            min_range, max_range = config.CLASSIFICATION_THRESHOLDS[predicted_class]
            print(f"预期价格变动: {min_range}% 到 {max_range}%")
            
            # 计算预期价格范围
            current_price = features_df['current_price'].iloc[0]
            expected_min = current_price * (1 + min_range / 100)
            expected_max = current_price * (1 + max_range / 100)
            print(f"预期价格范围: ${expected_min:.2f} 到 ${expected_max:.2f}")
        
        # 显示所有类别的概率
        print(f"\n=== 各类别概率分布 ===")
        all_probabilities = {}
        for class_label in sorted(config.CLASSIFICATION_THRESHOLDS.keys()):
            class_index = class_label + 4
            if 0 <= class_index < len(class_probabilities):
                prob = class_probabilities[class_index]
                all_probabilities[str(class_label)] = float(prob)
                
                # 获取该类别的价格区间
                min_val, max_val = config.CLASSIFICATION_THRESHOLDS[class_label]
                bar = "█" * int(prob * 20)
                print(f"  类别 {class_label:2d} ({min_val:5.1f}% 到 {max_val:5.1f}%): {prob:6.2%} |{bar:<20}|")
        
        # 生成人类可读的预测描述
        prediction_descriptions = {
            -4: "强烈看跌 (跌幅 > 5.5%)",
            -3: "中度看跌 (跌幅 2.5%-5.5%)",
            -2: "轻微看跌 (跌幅 1.0%-2.5%)",
            -1: "微弱看跌 (跌幅 0.5%-1.0%)",
             0: "中性走势 (涨跌幅度 ±0.5%)",
             1: "微弱看涨 (涨幅 0.5%-1.0%)",
             2: "轻微看涨 (涨幅 1.0%-2.5%)",
             3: "强烈看涨 (涨幅 > 2.5%)"
        }
        
        description = prediction_descriptions.get(predicted_class, "未知走势")
        print(f"\n📈 预测结论: {description}")
        
        # 风险提示
        if confidence < 0.6:
            print("⚠️  置信度较低，请谨慎参考")
        elif confidence > 0.8:
            print("✅ 置信度较高，预测可靠性强")
        else:
            print("ℹ️  置信度适中")
            
    except Exception as e:
        print(f"✗ 进行预测时出错: {e}")
        import traceback
        traceback.print_exc()
        
elif 'model_loaded' in locals() and not model_loaded:
    print("✗ 模型未加载，请先训练模型")
elif 'features_df' in locals():
    print("✗ 特征数据准备失败")
else:
    print("✗ 无法进行预测，缺少必要条件")

In [ ]:
# 预测结果可视化
print("正在生成预测结果可视化...")

if 'all_probabilities' in locals():
    try:
        # 创建概率分布图
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
        
        # 左图：柱状图
        classes = [int(k) for k in all_probabilities.keys()]
        probs = list(all_probabilities.values())
        
        colors = ['red' if x < 0 else 'green' if x > 0 else 'gray' for x in classes]
        bars = ax1.bar(classes, probs, color=colors, alpha=0.7)
        
        # 高亮预测类别
        if 'predicted_class' in locals():
            pred_idx = classes.index(predicted_class)
            bars[pred_idx].set_alpha(1.0)
            bars[pred_idx].set_edgecolor('black')
            bars[pred_idx].set_linewidth(2)
        
        ax1.set_xlabel('价格变动类别')
        ax1.set_ylabel('预测概率')
        ax1.set_title('各类别预测概率分布')
        ax1.set_xticks(classes)
        ax1.grid(True, alpha=0.3)
        
        # 右图：饼图
        sizes = probs
        labels = [f'类别 {c} ({p:.1%})' for c, p in zip(classes, probs)]
        ax2.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90, 
                colors=[plt.cm.Set3(i / len(classes)) for i in range(len(classes))])
        ax2.set_title('预测概率分布')
        
        plt.tight_layout()
        plt.show()
        
        # 显示近期价格走势
        if 'all_data' in locals() and len(all_data) > 0:
            recent_prices = [candle['close'] for candle in all_data[-48:]]  # 最近48小时
            timestamps = [datetime.fromtimestamp(candle['timestamp']/1000) for candle in all_data[-48:]]
            
            plt.figure(figsize=(12, 6))
            plt.plot(timestamps, recent_prices, marker='o', linewidth=1.5, markersize=3)
            plt.title('近期价格走势')
            plt.xlabel('时间')
            plt.ylabel('价格 ($)')
            plt.xticks(rotation=45)
            plt.grid(True, alpha=0.3)
            plt.tight_layout()
            plt.show()
        
    except Exception as e:
        print(f"✗ 生成可视化时出错: {e}")

In [ ]:
# 预测总结
print("=== 预测总结 ===")

if 'predicted_class' in locals():
    print(f"\n📊 预测详情:")
    print(f"  预测类别: {predicted_class}")
    print(f"  预测置信度: {confidence:.2%}")
    print(f"  当前价格: ${features_df['current_price'].iloc[0]:.2f}")
    
    if predicted_class in config.CLASSIFICATION_THRESHOLDS:
        min_range, max_range = config.CLASSIFICATION_THRESHOLDS[predicted_class]
        current_price = features_df['current_price'].iloc[0]
        expected_min = current_price * (1 + min_range / 100)
        expected_max = current_price * (1 + max_range / 100)
        
        print(f"  预期变动: {min_range}% 到 {max_range}%")
        print(f"  预期价格: ${expected_min:.2f} 到 ${expected_max:.2f}")
    
    print(f"\n🎯 预测结论: {prediction_descriptions.get(predicted_class, '未知走势')}")
    
    # 建议
    print(f"\n💡 操作建议:")
    if predicted_class > 0:
        if confidence > 0.7:
            print(f"  强烈建议: {description.split('(')[0].strip()} - 置信度高")
        else:
            print(f"  谨慎建议: {description.split('(')[0].strip()} - 置信度一般")
    elif predicted_class < 0:
        if confidence > 0.7:
            print(f"  强烈建议: {description.split('(')[0].strip()} - 置信度高")
        else:
            print(f"  谨慎建议: {description.split('(')[0].strip()} - 置信度一般")
    else:
        print(f"  建议: 价格走势中性，建议观望")
    
    print(f"\n⚠️  风险提示:")
    print(f"  - 加密货币市场波动较大，预测仅供参考")
    print(f"  - 建议结合其他技术指标和基本面分析")
    print(f"  - 请根据自身风险承受能力做出投资决策")
    
    print(f"\n✅ 预测完成时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
else:
    print("未能生成预测结果，请检查前面的步骤")
    
print(f"\n=== 工作流完成 ===")
print("整个预测流程已通过 notebook 完成，无需依赖 PriceMovementPredictor 类")